In [2]:
import os, shutil, torch, whisper as ws

print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())
print("ffmpeg on PATH:", bool(shutil.which("ffmpeg")))

Torch: 2.8.0+cpu CUDA: False
ffmpeg on PATH: True


In [7]:
audio_path = r"d:\Spark Mind\backend\audio\audio.mp3"
print("Audio exists:", os.path.exists(audio_path))

model = ws.load_model("base", device="cpu")
result = model.transcribe(audio_path, fp16=False)
print(result["text"])

Audio exists: True
 Ya, hijang yaakay kad morang para makapas
